In [1]:
from sc3.all import *

import random
import time
import numpy as np
from math import inf, log, sin, ceil
from functools import partial

from instruments import *
from fxs import *
from rhythms import *
from sc3_stuff import *


INFO:sc3.synth.server:setting global variable 's' to 'localhost'


In [2]:
restart()

INFO:sc3.synth.server:booting server 'localhost' on address 127.0.0.1:57110
INFO:SERVER.stdout:Number of Devices: 5
INFO:SERVER.stdout:   0 : "BlackHole 2ch"
INFO:SERVER.stdout:   1 : "External Headphones"
INFO:SERVER.stdout:   2 : "MacBook Air Microphone"
INFO:SERVER.stdout:   3 : "MacBook Air Speakers"
INFO:SERVER.stdout:   4 : "Multi-Output Device"
INFO:SERVER.stdout:
INFO:SERVER.stdout:"MacBook Air Microphone" Input Device
INFO:SERVER.stdout:   Streams: 1
INFO:SERVER.stdout:      0  channels 1
INFO:SERVER.stdout:
INFO:SERVER.stdout:"Multi-Output Device" Output Device
INFO:SERVER.stdout:   Streams: 1
INFO:SERVER.stdout:      0  channels 2
INFO:SERVER.stdout:
INFO:SERVER.stdout:SC_AudioDriver: sample rate = 48000.000000, driver's block size = 512
INFO:SERVER.stdout:SuperCollider 3 server ready.
INFO:sc3.synth._serverstatus:'localhost': requested registration id
INFO:sc3.synth._serverstatus:'localhost': setting client_id to 0


In [3]:
pan_mode = random.choice([0, 1])
note_count = 0

def play_notes_from_scale(scale, notes, synth=fm_2op, debug=False):
    global pan_mode, note_count
    dur = 0.2
    prints = []
    for i, n in enumerate(notes):
        edo_n = scale[n % len(scale)] + edo * (n // len(scale)) # TODO: use function
        prints.append(edo_n)
        freq = base_freq * (2 ** (edo_n / edo))
        if pan_mode == 0:
            pan_value = random.uniform(-1.0, 1.0)
        else:
            pan_value = sin(note_count / 3)
        synth(0, freq, dur=dur, pan=pan_value,
              add_action='head', amp=max(0.1, 0.4 - freq / 440 / 4))
        note_count += 1
        if note_count % 200 == 0:
            pan_mode = random.choice([0, 1])
    if debug:
        print('notes:', prints)

def play_and_wait(scale, notes, synth=fm_2op, dur=1.0):
    play_notes_from_scale(scale, notes, synth=synth)
    time.sleep(dur)

def arpeggiate_and_wait(scale, notes, synth=fm_2op, dur=0.25):
    for n in notes:
        play_notes_from_scale(scale, [n], synth=synth)
        time.sleep(dur)

def play_seq(seq):
    dur=0.2
    for n in seq:
        if type(n) is int:
            n = [n]
        if type(n) is not list:
            time.sleep(dur)
        else:
            play_and_wait(scale, n, dur=dur)

def play_seqs(scale, seqs, synths, dur=0.2):
    assert(len(seqs) == len(synths))
    max_idx = max([len(s) for s in seqs])
    for idx in range(max_idx):
        for j in range(len(seqs)):
            if len(seqs[j]) <= idx:
                continue
            n = seqs[j][idx]
            if type(n) is int:
                n = [n]
            if type(n) is not list:
                continue
            play_notes_from_scale(scale, n, synth=synths[j])
        time.sleep(dur)


In [4]:
def get_12edo(scale, edo, n):
    edo_n = scale[n % len(scale)] + edo * (n // len(scale)) # TODO: use function
    n_in_12_edo = edo_n / edo * 12
    return n_in_12_edo


def get_scale_steps(scale, edo, lo_ji, hi_ji):
    min_scale_steps = None
    max_scale_steps = None
    for i in range(1, edo):
        ratio = 2 ** (i / edo)
        if lo_ji <= ratio < hi_ji:
            if min_scale_steps is None:
                min_scale_steps = i
            max_scale_steps = i
        if hi_ji < ratio:
            break
#     print(min_scale_steps, max_scale_steps)
            
    scale_steps = []
    for step in range(1, len(scale) - 1):
        scale_with_step = list(scale[step:]) + [(n + edo) for n in scale[:step]]
        diffs = np.array(scale_with_step) - np.array(scale)
        mean_diff = np.mean(diffs)
#         print('mean', step, mean_diff)
        if min_scale_steps - 1 < mean_diff < max_scale_steps + 1:
            scale_steps.append(step)
#     print(scale_steps)
    return scale_steps

def get_semitones(scale, edo):
    return get_scale_steps(scale, edo, 24 / 25, 13 / 12)

def get_thirds(scale, edo):
    return get_scale_steps(scale, edo, 15 / 13, 17 / 13)

def get_fourths(scale, edo):
    return get_scale_steps(scale, edo, 14 / 11, 23 / 16)


def check_mos(edo, collection, min_step_in_cents=60, max_step_in_cents=350):
    collection_list = list(collection)
    collection_list.sort()
    collection_list.append(edo)
    diffs = set()
    for i in range(1, len(collection_list)):
        diff = collection_list[i] - collection_list[i - 1]
        diff_in_cents = diff * 1200 / edo
        if diff_in_cents < min_step_in_cents or diff_in_cents > max_step_in_cents:
            return False
        diffs.add(diff)
    if len(diffs) > 2:
        return False
    semitones = get_semitones(collection_list, edo)
    semitone_count = 0
    for i in range(1, len(collection_list)):
        diff = collection_list[i] - collection_list[i - 1]
        if diff in semitones:
            semitone_count += 1
            if semitone_count >= 2:
                return False
        else:
            semitone_count = 0
    return True

    
def gen_moses(edo, min_step_in_cents=60, max_step_in_cents=350,
              min_notes=5, max_notes_frac=0.6):
    moses = set()
    for step in range(1, edo):
        collection = set()
        cur = 0
        for k in range(int(max_notes_frac * edo + 1)):
            if cur in collection:
                break
            collection.add(cur)
            cur = (cur + step) % edo
            if len(collection) < min_notes:
                continue
            if len(collection) > max_notes_frac * edo:
                break
            if k > 1 and check_mos(edo, collection,
                                   min_step_in_cents=min_step_in_cents,
                                   max_step_in_cents=max_step_in_cents):
                notes = sorted(collection)
                for shift in range(edo):
                    mos = sorted([(n + shift) % edo for n in notes])
                    moses.add(tuple(mos))
    return list(moses)

def gen_scales(edo):
    return gen_moses(edo)

In [5]:
beat_count = 4
subbeat_count = 4
bpm = 120 # TODO

base_freq = 261.6255653006

all_edos = [12, 15, 16, 17, 19, 22, 23, 26, 27]
edo = random.choice(all_edos)
all_scales = gen_scales(edo)

print(edo, len(all_scales))
scale = random.choice(all_scales)

26 754


In [6]:
def find_nearest_note(scale, edo, cur, targets, prob=0.75, min_bound=None):
    residue = cur % len(scale)
    best_dist = None
    best_cand = None
    targets1 = list(set(targets))
    random.shuffle(targets1)
    for target in targets1:
        target_residue = target % len(scale)
        shifts = [-1, 0, 1]
        random.shuffle(shifts)
        for k in shifts:
            candidate = target - target_residue + residue + k * len(scale)
            if min_bound is not None and get_12edo(scale, edo, candidate) < min_bound:
                continue
            # FIXME: in theory we can get None for best_cand
            cur_dist = abs(candidate - target)
            if best_dist is None or (random.random() <= prob and cur_dist < best_dist):
                best_cand = candidate
                best_dist = cur_dist
        if best_cand is None:
            print('ERROR: best cand is None!')
            best_cand = candidate
    # todo: forbid voice crossings
    return best_cand

def smooth_chords(scale, edo, chords, start=0, min_bound=None, max_bound=None, prob=0.75):
    if start is None:
        prev_chord = [0]
    else:
        if type(start) is int:
            prev_chord = [start]
        else:
            prev_chord = start
    smoothed_chords = []
    for idx, chord in enumerate(chords):
        smoothed_chord = []
        for n in chord:
            smoothed_chord.append(find_nearest_note(scale, edo, n, prev_chord, min_bound=min_bound))
        
        # trying to place the root below other notes
        if (smoothed_chord[0] != min(smoothed_chord)) and (random.random() <= prob):
            if (random.random() <= 0.5):
                if (min_bound is None) or \
                        (get_12edo(scale, edo, smoothed_chord[0] - len(scale)) >= min_bound):
                    smoothed_chord[0] -= len(scale)
            else:
                if (max_bound is None) or \
                        (get_12edo(scale, edo, max(smoothed_chord[1:]) + len(scale)) <= max_bound):
                    smoothed_chord = [p + len(scale) for p in smoothed_chord]
                    smoothed_chord[0] -= len(scale)
        if max_bound is not None and\
                (get_12edo(scale, edo, max(smoothed_chord)) > max_bound):
            if (get_12edo(scale, edo, min(smoothed_chord) - len(scale)) > min_bound):
                smoothed_chord = [p - len(scale) for p in smoothed_chord]
        # todo: check the distance between notes
        smoothed_chords.append(smoothed_chord)
        prev_chord = smoothed_chord
    return smoothed_chords

def gen_chord_progression(scale, edo, tonic,
                          root_step=4, chord_step=2, chord_len=4, scheme=0,
                          prev_chord=None, min_bound=None, max_bound=None):
    roots = []
    root = tonic
    for i in range(beat_count):
        roots.append(root)
        root += root_step
    if scheme == 1:
        roots.reverse()
        roots = [roots[-1]] + roots[:-1]
    elif scheme == 2:
        roots.reverse()
    chords = []
    for root in roots:
        chord = []
        n = root
        for i in range(chord_len):
            chord.append(n)
            n += chord_step
        chords.append(chord)
    chords = smooth_chords(scale, edo, chords, start=prev_chord,
                           min_bound=min_bound, max_bound=max_bound)
    return chords
    # todo: add more schemes, e. g.:
    # i-i-ii-ii
    # i-i-i-ii
    # i-ii-i-ii
    # add slower changes, over several bars
    # ...


def gen_random_chord_progression(scale, edo, prev_chord=None, min_bound=None, max_bound=None):
    scheme = random.randint(0, 2)
    root_step = random.randint(1, len(scale) - 1)
    thirds = get_thirds(scale, edo)
    fourths = get_fourths(scale, edo)
    chord_step = random.choice(list(set(thirds + fourths)))
    chord_len = random.randint(3, 4)
#     modulate = random.choice([False, True])
    return gen_chord_progression(scale, edo, 0,
                                 root_step=root_step,
                                 chord_step=chord_step,
                                 chord_len=chord_len,
                                 scheme=scheme,
                                 prev_chord=prev_chord,
                                 min_bound=min_bound,
                                 max_bound=max_bound)
#                                  modulate=modulate)

# print(edo, scale)
# print(get_thirds(scale, edo))
# print(get_fourths(scale, edo))

In [7]:
def create_rhythm():
    rhythm = []
    for i in range(beat_count):
        note_count = random.randint(1, subbeat_count)
        subrhythm = euclidean(subbeat_count, note_count)
        shift = random.randint(0, subbeat_count - 1)
        subrhythm = subrhythm[shift:] + subrhythm[:shift]
        rhythm += subrhythm
    return rhythm

def gen_melody(scale, edo, chords, start=0, max_jump=None, max_rep=2, min_bound=None, max_bound=None):
    if max_jump is None:
        max_jump = random.choice([2, 3])
    total_count = beat_count * subbeat_count
    rhythm = create_rhythm()
    assert(len(rhythm) == total_count)
    assert(len(chords) == beat_count)
    note_idx = -1
    direction = 1
    last_note = start
    melody = []
    rep = 0
    for i in range(beat_count):
        for j in range(subbeat_count):
            if j == 0:
                available_notes = [p % len(scale) for p in chords[i]]
                if random.random() < 0.5:
                    direction *= -1
                if get_12edo(scale, edo, last_note) >= max_bound:
                    direction = -1
                if get_12edo(scale, edo, last_note) <= min_bound:
                    direction = 1
            else:
                available_notes = list(range(len(scale)))
            note_idx += 1
            if rhythm[note_idx] == 0:
                melody.append('.')
                continue
            candidates = []
            jump = -1
            while len(candidates) <= 1 or jump <= max_jump:
                jump += 1
                new_note = last_note + jump * direction
                if new_note % len(scale) in available_notes:
                    candidates.append(new_note)
            while True:
                new_note = random.choice(candidates)
                if new_note == last_note:
                    rep += 1
                    if rep > max_rep:
                        continue
                else:
                    rep = 0
                    last_note = new_note
                    break
            melody.append(last_note)
    return melody, last_note


def gen_chord_seq(scale, edo, chords):
    assert(len(chords) == beat_count)
    chord_seq = []
    schemes = [0, 1, 2, 3]
    random.shuffle(schemes)
    schemes = schemes[:2]
    for i in range(beat_count):
        chord = chords[i]
        scheme = random.choice(schemes)
        if scheme == 0:
            for j in range(subbeat_count):
                if j % 2 == 1:
                    chord_seq.append('.')
                else:
                    chord_seq.append([chord[0], random.choice(chord[1:])])
        elif scheme == 1:
            cur_seq = []
            for j in range(subbeat_count):
                cur_seq.append([])
            assert(scheme == 1)
            cur_seq[0].append(chord[0])
            for j in range(1, len(chord)):
                cur_seq[random.randint(0, subbeat_count - 1)].append(chord[j])
            for j in range(subbeat_count):
                if not cur_seq[j]:
                    cur_seq[j] = '.'
            chord_seq += cur_seq
        elif scheme == 2:
            interval = [chord[0], random.choice(chord[1:])]
            for j in range(subbeat_count):
                chord_seq.append(interval[j % len(interval)])
        else:
            assert(scheme == 3)
            for j in range(subbeat_count):
                chord_seq.append(chord[j % len(chord)])
    return chord_seq


In [8]:
def convert_to_edo(scale, edo, notes):
    edo_notes = []
    for n in notes:
        edo_n = scale[n % len(scale)] + edo * (n // len(scale))
        edo_notes.append(edo_n)
    return edo_notes

def approximate_note(scale, edo, n, old_scale, old_edo):
    n_in_old_edo = convert_to_edo(old_scale, old_edo, [n])[0]
    octave = n_in_old_edo // old_edo
    n_in_edo = n_in_old_edo / old_edo * edo
    nearest = None
    best_dist = None
    for n2 in range(len(scale)):
        n2_in_octave = n2 + octave * len(scale)
        n2_in_octave_in_edo = convert_to_edo(scale, edo, [n2_in_octave])[0]
        cur_dist = abs(n2_in_octave_in_edo - n_in_edo)
        if nearest is None or cur_dist < best_dist:
            nearest = n2_in_octave
            best_dist = cur_dist
    return nearest


def modulate_and_recalc(scale, edo, chord, root):
    chord_in_edo = []
    for n in chord:
        chord_in_edo.append(scale[n % len(scale)])
    # FIXME: don't know why but this alternative fails:
    # chord_in_edo = convert_to_edo(scale, edo, chord)
    # chord_in_edo = [n % edo for n in chord_in_edo]
    # probably because of negative values, and using % operator
    available_scales = []
    for cand_scale in all_scales:
        is_good = True
        for n in chord_in_edo:
            if n not in cand_scale:
                is_good = False
                break
        if is_good:
            available_scales.append(cand_scale)
    if scale not in available_scales:
        print('ERROR: chord mismatches scale', scale, chord_in_edo)
    assert(scale in available_scales)
    new_scale = random.choice(available_scales)
    if new_scale == scale:
        new_root = (root + random.randint(1, len(scale) - 1)) % len(scale)
    else:
        new_root = random.randint(0, len(scale) - 1)
    return new_scale, new_root


In [9]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

def print_cut(*inputs):
    wrap_len=70
    s = ''
    for inp in inputs:
        s += str(inp) + ' '
    s = s.split(' ')
    batch = ''
    for part in s:
        if len(batch) + 1 + len(part) > wrap_len:
            print(batch + ' ...')
            batch = '... ' + part
        else:
            batch += ' ' + part
    if batch:
        print(batch)

prev_chord = [-len(scale)]
root = random.randint(0, len(scale) - 1)
start_note = root
start_note2 = root

def get_random_bass(edo):
    return partial(fm_2op,
                   m_ratio=random.uniform(1.0, 1.0),
                   c_ratio=random.uniform(1.0, 1.0),
                   index=random.choice([0, 1, 2, 3]),
                   i_scale=random.randint(1, 5),
                   decay=0.3)

dur = 0.12
note_count = 0
beat_count = 4
subbeat_count = 4
edo = 12
all_scales = gen_scales(edo)
scale = random.choice(all_scales)

bass = get_random_bass(edo)
solo = random.choice([chiptune_saw, chiptune_triangle])
solo2 = random.choice([chiptune_saw, chiptune_triangle])

counter = 0
print('edo', edo)
print('scale', scale)
start = time.perf_counter()
last_edo_change = time.perf_counter()
edo_change_period = random.randint(20, 60)
last_instrument_switch = time.perf_counter()
while time.perf_counter() - start < 18*60 + 30:
    chords = gen_random_chord_progression(scale, edo, prev_chord=prev_chord, min_bound=-18, max_bound=-6)
    prev_chord = chords[-1]
    melody, last_note = gen_melody(scale, edo, chords, start=start_note, min_bound=-12, max_bound=18)
    start_note = last_note
    melody2, last_note2 = gen_melody(scale, edo, chords, start=start_note2, min_bound=-20, max_bound=-8)
    start_note2 = last_note2
    chord_seq = gen_chord_seq(scale, edo, chords)
    print_cut('chords', chord_seq)
    print_cut('melody', melody)
    print_cut('melody2', melody2)
    switch_to_new_edo = False
    if time.perf_counter() - last_edo_change > edo_change_period:
        switch_to_new_edo = True
    modulate = False
    if not switch_to_new_edo:
        modulate = random.random() < 0.33
    reps = 1
    if not modulate and not switch_to_new_edo:
        reps = random.randint(1, 4)
    switch_instruments = random.random() < 0.2
    if time.perf_counter() - last_instrument_switch > 30:
        switch_instruments = True
    for j in range(reps):
        counter += 1
        play_seqs(scale, [chord_seq, melody, melody2], [bass, solo, solo2], dur=dur)
        if switch_instruments:
            last_instrument_switch = time.perf_counter()
            print('Changing instruments and tempo')
            bass = get_random_bass(edo)
            solo = random.choice([chiptune_saw, chiptune_triangle])
            solo2 = random.choice([chiptune_saw, chiptune_triangle])
            dur = random.uniform(0.12, 0.35)
            beat_count = random.randint(4, 5)
            subbeat_count = random.randint(3, 5)
    if modulate:
        print('modulating')
        new_edo = edo
        new_scale, new_root = modulate_and_recalc(scale, edo, chords[-1], root)
    elif switch_to_new_edo:
        edo_change_period = random.randint(20, 60)
        last_edo_change = time.perf_counter()
        new_edo = edo
        while new_edo == edo:
            new_edo = random.choice(all_edos)
        all_scales = gen_scales(new_edo)
        new_scale = random.choice(all_scales)
        new_root = random.randint(0, len(new_scale) - 1)
        printmd('# New EDO: ' + str(edo) + ' -> ' + str(new_edo))
    if modulate or switch_to_new_edo:
        start_note = approximate_note(new_scale, new_edo, start_note, scale, edo)
        start_note2 = approximate_note(new_scale, new_edo, start_note2, scale, edo)
        prev_chord_modulated = []
        for n in prev_chord:
            prev_chord_modulated.append(approximate_note(new_scale, edo, n, scale, edo))
        prev_chord = prev_chord_modulated
        edo = new_edo
        scale = new_scale
        root = new_root
        print('edo', edo)
        print('scale', scale)
printmd('# FIN!')

edo 12
scale (0, 2, 4, 5, 7, 9, 11)
 chords [-8, -4, -8, -4, [-10], '.', [-5], [-6, -9], -5, -4, -5, -4, ...
... [-7], '.', [-9], [-3, -6]] 
 melody [6, '.', 8, '.', '.', 11, '.', '.', 10, 8, '.', 7, 4, '.', 1, ...
... '.'] 
 melody2 [-1, '.', -2, '.', -6, '.', -9, '.', -12, '.', '.', '.', '.', ...
... -11, '.', '.'] 
modulating
edo 12
scale (0, 2, 4, 5, 7, 9, 10)
 chords [-9, -8, -9, -8, -6, -8, -6, -8, [-10, -9], [-5], '.', [-6], ...
... -7, -9, -7, -9] 
 melody [3, 6, 7, 10, '.', 12, 13, 14, '.', '.', 13, '.', 12, 9, 6, ...
... '.'] 
 melody2 [-12, -13, -16, '.', -15, '.', '.', '.', '.', -13, '.', -10, ...
... -13, -15, -17, -19] 
 chords [-7, -8, -7, -8, [-9], [-5], [-7, -10], '.', -4, -9, -4, -9, ...
... -6, -9, -6, -9] 
 melody [9, 11, 12, '.', '.', '.', 10, '.', '.', 13, '.', 16, 15, 13, ...
... 10, '.'] 
 melody2 [-17, '.', -15, '.', -12, '.', '.', '.', -11, -10, -8, -5, ...
... '.', '.', '.', -7] 
Changing instruments and tempo
modulating
edo 12
scale (0, 2, 3, 5, 7, 9, 10)
 c

# New EDO: 12 -> 16

edo 16
scale (1, 5, 8, 11, 14)
 chords [[-4, -2], '.', [0], [-3, -6], '.', [-3, -4], [-7, -5, -3], ...
... '.', '.', [-6], [-2], [-4], [-5], '.', [-3, -6]] 
 melody [-2, '.', -4, '.', '.', -6, -5, '.', -4, '.', -7, -10, -8, ...
... '.', -6] 
 melody2 [-7, -8, '.', '.', -6, '.', -5, '.', '.', -6, -8, -10, -8, ...
... '.', '.'] 
 chords [-5, -3, -6, -6, -7, -6, -7, -5, -7, -8, -6, -4, -4, -7, -4] 
 melody [-5, -4, 0, 1, 2, 4, 3, '.', 0, '.', '.', 4, 8, 12, 14] 
 melody2 ['.', '.', -6, '.', -9, '.', '.', -6, '.', -3, -2, '.', -5, ...
... -8, -9] 
 chords [[-5, -4], '.', [-5, -4], -6, -5, -6, -7, -5, -7, -8, -7, -8, ...
... -4, -7, -4] 
 melody [10, '.', 8, '.', 7, 5, 9, 12, 14, '.', 10, 7, '.', 6, 2] 
 melody2 [-8, '.', '.', '.', '.', -10, -8, -5, -3, '.', -4, -5, -7, ...
... '.', -8] 
modulating
edo 16
scale (1, 5, 8, 11, 14)
 chords [[-7, -3], '.', [-5, -6], [-4, -3], [-7], [-5], [-6, -4, -5], ...
... '.', [-7], -8, -6, -8, -5, -4, -5] 
 melody ['.', -2, -5, -4, '.', 0, -4, '.', -7, '.'

# New EDO: 16 -> 22

edo 22
scale (3, 5, 11, 13, 19)
 chords [[-8], '.', '.', '.', [-6, -4, -7], [-7], '.', '.', '.', [-5, ...
... -8, -6], [-6], '.', '.', [-4, -5], [-7], [-5], '.', '.', [-3], ...
... [-6, -4]] 
 melody [1, '.', 0, '.', -2, -3, -6, -9, -12, -13, -11, '.', -10, '.', ...
... -8, -6, '.', '.', '.', '.'] 
 melody2 [-9, '.', -13, '.', -16, -15, '.', '.', '.', '.', -11, -8, ...
... -6, -3, -2, -4, -5, '.', -6, '.'] 
modulating
edo 22
scale (1, 3, 5, 9, 11, 13, 15, 19, 21)
 chords [[-9, -7], '.', '.', '.', [-5], [-6, -4, -11], '.', '.', '.', ...
... '.', [-12, -10], '.', [-8], '.', '.', [-9, -7], '.', [-9, -14], ...
... '.', [-9, -7]] 
 melody [-9, '.', -8, -6, '.', '.', -8, '.', '.', '.', -12, -15, -19, ...
... -21, -24, -18, -16, -14, -10, -8] 
 melody2 [-9, -7, -6, -3, -1, -4, '.', -7, -10, -11, '.', -12, '.', ...
... -14, -17, -16, '.', -14, -11, -10] 
 chords [-9, -7, -9, -7, -9, -13, -11, -9, -7, -13, -8, -11, -8, -11, ...
... -8, -12, -10, -8, -6, -12] 
 melody [-7, '.', -5, '.', '.', '.'

...

edo 12
scale (1, 2, 4, 6, 7, 9, 11)
 chords [[-6, -4], '.', '.', [-7], [-10], [-8], [-9], '.', [-5, -6], ...
... '.', [-10], [-7], [-4, -8], '.', '.', -5, -9, -6, -10, -5, [-7], ...
... '.', [-8, -5], [-4], '.'] 
 melody [0, -1, '.', -4, '.', '.', -3, '.', '.', -1, -3, -5, '.', -6, ...
... -8, -5, '.', -4, -3, -1, '.', '.', '.', -2, '.'] 
 melody2 ['.', -3, '.', -5, -8, '.', -11, -14, -15, -16, '.', -15, ...
... '.', '.', -13, -12, -8, '.', -4, '.', '.', -8, -9, '.', -11] 
modulating
edo 12
scale (1, 4, 6, 8, 11)
 chords [-5, -4, -3, -2, -5, [-1], [-5], '.', [-4], [-3], [-2], [-4], ...
... [-1], [-5], '.', -8, -2, -1, -5, -8, [-4, -2], [-8], [-1], '.', ...
... '.'] 
 melody [1, '.', 5, '.', '.', 9, '.', 11, '.', 15, '.', 11, 9, 7, 3, ...
... '.', 2, '.', '.', '.', '.', '.', '.', '.', -2] 
 melody2 ['.', -4, '.', '.', 0, -1, -4, '.', -8, -11, -7, -6, -3, 0, ...
... 3, '.', 1, -2, '.', -6, -7, -10, '.', -13, '.'] 
modulating
edo 12
scale (1, 3, 4, 6, 8, 9, 11)
 chords [-7, -3, 1, -7, -3,

# New EDO: 12 -> 17

edo 17
scale (0, 3, 6, 9, 11, 14)
 chords [[-6, -8], '.', [-6, -8], '.', -3, -7, -3, -7, [-6, -8], '.', ...
... [-6, -8], '.', [-9, -7], '.', [-9, -5], '.'] 
 melody [-4, -6, -10, -13, '.', '.', '.', -10, '.', -8, '.', -5, '.', ...
... '.', -4, '.'] 
 melody2 [-10, '.', -7, -5, -7, '.', -9, -11, -10, -8, -5, -4, '.', ...
... -6, '.', -9] 
 chords [-6, -4, -6, -4, -4, -2, -4, -2, [-8], [-4], [0], '.', [-6], ...
... '.', [-8], [-4]] 
 melody [0, 3, 6, 9, '.', 7, '.', 5, 2, -2, -6, -8, -6, '.', -5, -3] 
 melody2 ['.', '.', '.', -10, -8, '.', -6, -5, '.', '.', -8, '.', -10, ...
... -12, -13, -15] 
Changing instruments and tempo
 chords [-6, -3, -6, -3, -6, -3, -6, -6, -4, -7, -4, -4, -8, -8, -8, ...
... -8] 
 melody [0, 2, 4, 5, 3, '.', 2, '.', -1, '.', '.', '.', -2, '.', -4, ...
... '.'] 
 melody2 ['.', -13, -12, -9, -6, -4, -1, 2, '.', 0, -2, -4, -5, '.', ...
... '.', '.'] 
modulating
edo 17
scale (0, 3, 6, 8, 11, 14)
 chords [-3, -6, -3, -6, [-2, -5], [-5, -2], '.', '.', [-7], '.', [-4,

# FIN!

INFO:sc3.synth.server:server 'localhost' exited with exit code -9
